user_id: 사용자의 고유 ID <br>
movie_id: 영화의 고유 ID<br>
rating: 사용자가 영화에 준 평점<br>
timestamp: 평점이 기록된 시간<br>

In [1]:
#별점 1,2,3, 별로 카운트

In [2]:
from pyspark import SparkConf, SparkContext
from collections import OrderedDict
conf = SparkConf().setMaster("local").setAppName("241204_02_MovieLens")
sc = SparkContext(conf=conf)

24/12/06 18:04:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
import os
directory = os.path.join(os.getcwd(), "data")  # 현재 디렉토리의 data 폴더
filename  = "u.data"
file_path = os.path.join(directory, filename)

In [4]:
lines = sc.textFile("file:///" + file_path.replace("\\", "/"))

In [5]:
lines.collect()[:5]

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [6]:
ratings = lines.map(lambda x: x.split('\t')[2])
ratings.collect()[:5]

['3', '3', '1', '2', '1']

In [7]:
result = ratings.countByValue()
result

defaultdict(int, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

In [8]:
#3점이상인 영화의 평균 평점 계산하기

In [9]:
#전체 파싱 > 3점이상 데이터 추출
high_rating_rdd = lines.map(lambda x:x.split()).filter(lambda row: int(row[2]) >= 3)
high_rating_rdd.take(5)


[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['298', '474', '4', '884182806'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817']]

In [10]:
#영화별 평균 평점 계산

In [11]:
movie_ratings = high_rating_rdd.map(lambda x: (x[1], (int(x[2]), 1)))
movie_ratings.take(3)

[('242', (3, 1)), ('302', (3, 1)), ('474', (4, 1))]

In [12]:
#key(id) 별로 3점,4점,5점 합산, 카운트 누적
movie_total = movie_ratings.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
movie_total.take(3)

[('242', (453, 108)), ('302', (1214, 285)), ('474', (813, 188))]

In [13]:
#평균
movie_avg = movie_total.mapValues(lambda x: x[0]/x[1])
movie_avg.take(3)

[('242', 4.194444444444445),
 ('302', 4.2596491228070175),
 ('474', 4.324468085106383)]

In [14]:
sc.stop()